In [ ]:
import pandas as pd 
import numpy as np
from scipy.sparse import load_npz,save_npz,lil_matrix,csr_matrix
from tqdm import tqdm 
import os

In [ ]:
## y sparse generation for the model training in step 3 
# idea is to identify the corresponding aux tasks and associate their performance in the image-based model

In [ ]:
path = '../notebooks/aux_data/baseline_plus_aux_data/matrices/cls/cls_T10_y.npz'
y = load_npz(path)

In [ ]:
y.shape

In [ ]:
path = './cp/summary_eps_0.05.csv'
df_cp = pd.read_csv(path)

In [ ]:
path = '../notebooks/aux_data/baseline_plus_aux_data/results_tmp/classification/T8c.csv'
t8c = pd.read_csv(path)

In [ ]:
t8c['cont_classification_task_id'].max()

In [ ]:
path = './mapping/baseline_image_model_task_mapping.csv'
baseline_imagemodel_mapping = pd.read_csv(path)

In [ ]:
baseline_imagemodel_mapping

In [ ]:
df_cp['index'].max()

In [ ]:
df_cp_mgd = pd.merge(
    df_cp
    ,baseline_imagemodel_mapping
    ,left_on='index'
    ,right_on='cont_classification_task_id_image'
    ,how='inner'
)

In [ ]:
path = './cp/summary_eps_0.05_mgd_cp.csv'
df_cp_mgd.to_csv(path, index=False)

In [ ]:
## get to the baseline+aux cctis 

In [ ]:
### df_cp_mgd vs df_mgd_cp !! TO CHECK 

In [ ]:
df_matching = pd.merge(
    df_mgd_cp
    ,t8c
    ,on=['input_assay_id','threshold']
    ,how='inner'
    ,suffixes=('_image','_baselineaux')
)

In [ ]:
df_matching['cont_classification_task_id_baselineaux'].hist()

In [ ]:
y_lil = lil_matrix(y)

In [ ]:
all_cctis_baselineaux = set(t8c['cont_classification_task_id'].dropna().unique())
base_path = './aux_data/baseline_plus_aux_data/matrices/cls/confidence_selection/'

In [ ]:
len(all_cctis_baselineaux)

In [ ]:
file_core = 'ppv{}_npv{}'.format(l1,l2).replace('.','_')
y_out = y_lil.copy()

In [ ]:
all_cctis_aux = set(df_matching['cont_classification_task_id_baselineaux'])

In [ ]:
cctis_to_keep = set(df_matching.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id_baselineaux'])

In [ ]:
len(all_cctis_aux)

In [ ]:
len(cctis_to_keep)

In [ ]:
cctis_to_drop = all_cctis_aux.difference(cctis_to_keep)

In [ ]:
len(cctis_to_drop)

In [ ]:
lst = [0
       ,0.2
       ,0.4
       ,0.5
       ,0.6
       ,0.7
       ,0.8
       ,0.9
       ,0.95
       ,0.99
      ]

In [ ]:
file_core = 'ppv{}_npv{}'.format(l1,l2).replace('.','_')
y_out = y_lil.copy()
all_cctis_aux = set(df_matching['cont_classification_task_id_baselineaux'])


for i,l1 in tqdm(enumerate(lst)):
    l2 = l1
    #for j,l2 in enumerate(lst): 
    file_core = 'ppv{}_npv{}'.format(l1,l2).replace('.','_')
    y_out = y_lil.copy()
    cctis_to_keep = set(df_matching.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id_baselineaux'])
    cctis_to_drop = all_cctis_aux.difference(cctis_to_keep)        
    for ccti in cctis_to_drop: 
        y_out[:,ccti] = 0
    y_out_csr = csr_matrix(y_out)
    print(y_out_csr.shape)
    save_npz(os.path.join(base_path,'cls_T10_y_'+file_core+'.npz')
             ,y_out_csr)

In [ ]:
# baseline y-sparse creation
# approach : set all aux tasks to 0 

In [ ]:

file_core = 'baseline'
y_out = y_lil.copy()



for ccti in all_cctis_aux: 
    y_out[:,ccti] = 0
y_out_csr = csr_matrix(y_out)
print(y_out_csr.shape)
save_npz(os.path.join(base_path,'cls_T10_y_'+file_core+'.npz')
         ,y_out_csr)